In [1]:
from pushshift_py import PushshiftAPI
import datetime as dt
import psaw

You can also use directly PushishiftAPI() without psaw.

In [2]:
api = psaw.PushshiftAPI()

In [3]:
startEpoch = int(dt.datetime(2020,1,1).timestamp())

In [4]:
startEpoch

1577858400

The following block shows how we can get information using pushshift. It shows how we can specify the features and get them. The returned data type is a generator with "submission" type as elements, though we can certainly make them into a list.

In [10]:
features = ['url','author', 'title', 'subreddit', 'id', 'created', 'score']
subreddit = 'NBA'

data = api.search_submissions(after=startEpoch,
                            subreddit=subreddit,
                            filter= features,
                            limit=10)

for datum in data:
    print(datum.id, datum.subreddit, datum.title, datum.author, datum.url, datum.created, datum.score)

KeyboardInterrupt: 

In [6]:
import praw

reddit = praw.Reddit(
    client_id="kxbUr-4PyE7DlQ",
    client_secret="Q5rIAPS9IHZ1QgOIkHNY09Y9VMxDsA",
    password="AACAXZDE",
    user_agent="testscript by u/kc_the_scraper",
    username="kc_the_scraper",
)

We can use praw to get the post body using the following block.

In [7]:
reddit.submission(id='eiev5d').selftext

'first off, happy new years. feel free to spam this with “new years shit post” and get your upvotes. anyways,\n\n ben and embiid won’t win together. ever. if ben simmons got built around like giannis did, aka shooters all around him, as well as a coach that would push him to shoot 1-2 threes per game, that said team would be unstoppable and he would be great.\n\nyes, ben is a coward. but, the great sub known as r/nba, i am proposing you the idea that if ben had shooters around him he would be amazing. imagine him driving inside and using his fantastic passing ability to kick it to shooters. it would be great. instead, embiid is in the paint, and he won’t shoot it, and it’s just a mess.\n\na trade idea i thought if could be ben for dlo, once ben’s max kicks in, so next season. (we’ll see how this one plays out) ben simmons would be playing with klay and steph, could maybe play that forward role, and he would be golden. him with his passing ability and steph and klay with their shooting 

In the following blocks, we create tables and store the information. For some reason, though, the api often acts up and freezes when we loop through the data.

In [4]:
import sqlite3

In [5]:
conn = sqlite3.connect('redditPosts.sqlite')
cur = conn.cursor()

In [69]:
cur.execute('''CREATE TABLE IF NOT EXISTS Posts(
                id TEXT PRIMARY KEY,
                subreddit TEXT,
                title TEXT,
                author TEXT,
                url TEXT,
                created int)
                ''')

In [7]:
features = ['url','author', 'title', 'subreddit', 'id', 'created']
subreddit = 'stock'
latest = dt.datetime(2021,5,8).timestamp()
earliest = dt.datetime(2020,1,1).timestamp()

startEpoch = earliest

while startEpoch <= latest:
    data = api.search_submissions(after=startEpoch,
                            subreddit=subreddit,
                            filter= features,
                            limit=100)
    
    for datum in data:
        print('Got here 2.')
        cur.execute('''INSERT OR IGNORE INTO Posts VALUES (?,?,?,?,?,?)'''
                    , (datum.id, datum.subreddit, datum.title, datum.author, datum.url, datum.created))
        
        currentTime = datum.created
    
    conn.commit()
    if currentTime == startEpoch:
        break
    startEpoch = currentTime + 1
    print(dt.datetime.fromtimestamp(startEpoch))    



KeyboardInterrupt: 